Coursera IBM Capstone
==
## Segmenting and Clustering Neighborhoods in Toronto
### By: Ted Hartnell
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Step 1: Load Dependencies

In [1]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np

In [2]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import the Python Wikipedia Crawl library
import wikipedia
wikipedia.summary("List of postal codes of Canada: M", sentences=2)

'This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area.'

In [4]:
# Import BeautifulSoup to parse the crawled HTML
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

In [5]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import the Geocoder Library to lookup Latitude and Longitude of Toronto Neighborhoods
import geocoder

In [7]:
# Import library to handle REST requests
import requests

In [8]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Import Map Rendering Library
import folium

In [10]:
# Have the user manually enter the Foursquare Credentials so they are not stored in the raw Jupyter notebook
# Clear Cell Outputs after running!!!
FOURSQUARE_CLIENT_ID = input("[1/3] Enter your FourSquare Client ID:")
FOURSQUARE_CLIENT_SECRET = input("[2/3] Enter your FourSquare Client Secret:")
FOURSQUARE_VERSION = input("[3/3] Enter your FourSquare Version Number:")

## Step 2: Crawl Wikipedia Toronto Neighborhoods

In [11]:
# Get the link to the 'List of Neighborhoods in Toronto' Wikipedia Page and test it
wikipediaNeighborhoods = wikipedia.page("List_of_postal_codes_of_Canada:_M")
print( wikipediaNeighborhoods.title )
print( wikipediaNeighborhoods.url )

List of postal codes of Canada: M
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [12]:
# Get the HTML content from the 'List of Neighborhoods in Toronto' Wikipedia Page as a String
htmlNeighborhoods = wikipediaNeighborhoods.html()

# Check the HTML content was collected
print( htmlNeighborhoods[0:100] )

<div class="mw-parser-output"><p>This is a list of <a href="/wiki/Postal_codes_in_Canada" title="Pos


In [13]:
# Parse the HTML so it can be searched by BeautifulSoup
parsedNeighborhoods = BeautifulSoup(htmlNeighborhoods)

In [14]:
# Find the list of Neighborhoods at the bottom of the 'List of Neighborhoods in Toronto' Wikipedia Page
find_allNeighborhoods = parsedNeighborhoods.find('table', class_='wikitable').find_all('tr', class_='')
print('Found {} potential Neighborhoods in the HTML'.format( len(find_allNeighborhoods) ))

# Check that the first 5 neighborhoods contains useful data
find_allNeighborhoods[:5]

Found 289 potential Neighborhoods in the HTML


[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>]

In [15]:
# Define the dataframe columns for the Neighborhoods
columnsNeighborhoods = ['PostalCode', 'Borough', 'Neighborhood', 'URL', 'Title', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=columnsNeighborhoods)

In [16]:
# Try reading in the neighborhoods from earlier - necessary as it takes a very long time to parse neighborhood data
neighborhoods = pd.read_csv("Toronto Neighborhoods 013 Tab-Delineated.csv", sep='\t')
neighborhoods.drop(neighborhoods.columns[0], axis=1, inplace=True) # Drop the ID column

In [17]:
# Convert crawled neighborhood HTML data into structured DataFrames
# Note that this routine can be run several times to retry latitude/longitude lookups as they very often fail

# Limit the number of neighborhoods that can be added each time this cell is run
MAX_NEIGHBORHOODS = 300
countNeighborhoods = 0

for row in find_allNeighborhoods:
    
    countNeighborhoods += 1
    if countNeighborhoods > MAX_NEIGHBORHOODS:
        break
    
    # Get the list of rows found within each crawled table fragment
    tag = row.findAll('td')

    # Extract the Postal Code
    try:
        postalcodeNeighborhood = tag[0].get_text().strip()
    except:
        print("Row {} doesn't have any Neighborhood information - Skip".format(countNeighborhoods))
        continue
    
    # Extract the Borough - if the Borough is 'Not assigned' then skip the row
    try:
        boroughNeighborhood = tag[1].get_text().strip()
    except:
        print("Row {} doesn't have a Borough - Skip".format(countNeighborhoods))
        continue

    if boroughNeighborhood == 'Not assigned':
        print("Row {} Borough is marked as 'Not Assigned' - Skip".format(countNeighborhoods))
        continue
        
    # Extract the Name of the Neighborhood and clean it up
    try:
        nameNeighborhood = tag[2].get_text().strip()
    except:
        print("Row {} doesn't have a Neighborhood - Skip".format(countNeighborhoods))
        continue

    nameNeighborhood = nameNeighborhood.replace(", Toronto","").replace(" (Toronto)","").replace(" (neighbourhood)","").replace(" (page does not exist)","")
    
    # If the Neighborhood Name is 'Not assigned' then use the Borough Name instead
    if nameNeighborhood == 'Not assigned':
        nameNeighborhood = boroughNeighborhood

    # Extract the URL for the Neighborhood (or Borough)
    try:
        urlNeighborhood = tag[2].a['href'].strip() # Try getting the URL for the Neighborhood
    except:
        try:
            urlNeighborhood = tag[1].a['href'].strip() # Else try getting the URL for the Borough
        except:
            urlNeighborhood = ''

    # Extract the Title for the Neighborhood - the Title often contains more information so is used for looking up the Latitude and Longitude
    try:
        titleNeighborhood = tag[2].a['title'].strip() # Try getting the Title for the Neighborhood
    except:
        try:
            titleNeighborhood = tag[1].a['title'].strip() # Else try getting the Title for the Borough
        except:
            titleNeighborhood = ''

    titleNeighborhood = titleNeighborhood.replace(", Toronto","").replace(" (Toronto)","").replace(" (neighbourhood)","").replace(" (page does not exist)","")
    titleNeighborhood = titleNeighborhood + ', Toronto, Ontario' # Use the Neighborhood Title to find the latitude and longitude
            
    # Check that the Neighborhood Name being added to the DataFrame is unique - if the Name is not unique then it has already been added to the DataFrame (recall that this routine can be run many times in order to ensure all location data is loaded)
    if nameNeighborhood in neighborhoods["Neighborhood"].tolist():
        print('The Neighborhood of {} has already been loaded into the DataFrame'.format(nameNeighborhood))
        continue
    
    latitudeNeighborhood = ''
    longitudeNeighborhood = ''

    # (optional) Lookup the Latitude and Longitude of the Neighborhood
    if True:
        try:
            locationNeighborhood = geocoder.arcgis('{}, Toronto, Ontario'.format(nameNeighborhood))
            latitudeNeighborhood = locationNeighborhood.latlng[0]
            longitudeNeighborhood = locationNeighborhood.latlng[1]
            print('{}: The geograpical coordinate of {} is {}, {}.'.format(postalcodeNeighborhood, titleNeighborhood, latitudeNeighborhood, longitudeNeighborhood))
        except:
            print('{}: The geograpical coordinates of {} failed'.format(postalcodeNeighborhood, titleNeighborhood))
            latitudeNeighborhood = ''
            longitudeNeighborhood = ''
            continue

    print('{}: Adding Neighborhood {}'.format(postalcodeNeighborhood, titleNeighborhood))
    neighborhoods = neighborhoods.append({
                                            'PostalCode': postalcodeNeighborhood,
                                            'Borough': boroughNeighborhood,
                                            'Neighborhood': nameNeighborhood,
                                            'URL': urlNeighborhood,
                                            'Title': titleNeighborhood,
                                            'Latitude': latitudeNeighborhood,
                                            'Longitude': longitudeNeighborhood
                                            }, ignore_index=True)

# Note that not all of the locations of the Neighborhood names from the Wikipedia page can be found by the geolocator
print('Finished adding {} Neighborhoods from {} crawled rows.'.format(len(neighborhoods), len(find_allNeighborhoods), ))

Row 1 doesn't have any Neighborhood information - Skip
Row 2 Borough is marked as 'Not Assigned' - Skip
Row 3 Borough is marked as 'Not Assigned' - Skip
The Neighborhood of Parkwoods has already been loaded into the DataFrame
The Neighborhood of Victoria Village has already been loaded into the DataFrame
M5A: The geograpical coordinate of Harbourfront, Toronto, Ontario is 43.63950995556605, -79.38315993782102.
M5A: Adding Neighborhood Harbourfront, Toronto, Ontario
The Neighborhood of Regent Park has already been loaded into the DataFrame
The Neighborhood of Lawrence Heights has already been loaded into the DataFrame
The Neighborhood of Lawrence Manor has already been loaded into the DataFrame
The Neighborhood of Queen's Park has already been loaded into the DataFrame
Row 11 Borough is marked as 'Not Assigned' - Skip
The Neighborhood of Islington Avenue has already been loaded into the DataFrame
The Neighborhood of Rouge has already been loaded into the DataFrame
The Neighborhood of Ma

In [18]:
# Check that the neighborhoods have been loaded correctly
neighborhoods.head()

PostalCode           Borough      Neighborhood                     URL  \
0        M3A        North York         Parkwoods         /wiki/Parkwoods   
1        M4A        North York  Victoria Village  /wiki/Victoria_Village   
2        M5A  Downtown Toronto       Regent Park       /wiki/Regent_Park   
3        M6A        North York  Lawrence Heights  /wiki/Lawrence_Heights   
4        M6A        North York    Lawrence Manor    /wiki/Lawrence_Manor   

                                Title   Latitude  Longitude  
0         Parkwoods, Toronto, Ontario  43.686575 -79.409993  
1  Victoria Village, Toronto, Ontario  43.731540 -79.314280  
2       Regent Park, Toronto, Ontario  43.660690 -79.360310  
3  Lawrence Heights, Toronto, Ontario  43.723570 -79.437110  
4    Lawrence Manor, Toronto, Ontario  43.722920 -79.431310

In [19]:
# Save the neighborhoods as a tab-delineated CSV file
neighborhoods.to_csv("Toronto Neighborhoods 013 Tab-Delineated.csv", sep='\t')

In [20]:
# Get the Location of the geographic center of Toronto to center the map
nameLocation = "Leaside, Toronto, Ontario" # The geographic center is not "Toronto, Canada"
locationToronto = geocoder.arcgis(nameLocation)
print('The geograpical coordinate of the city center at {} is {}, {}.'.format(nameLocation, locationToronto.latlng[0], locationToronto.latlng[1]))

The geograpical coordinate of the city center at Leaside, Toronto, Ontario is 43.70023937855272, -79.3510651247247.


In [21]:
# Create map of Toronto using {latitude, longitude} values and Neighborhood label
mapToronto = folium.Map(location=[locationToronto.latlng[0], locationToronto.latlng[1]], zoom_start=11)

# Add markers to Map
for latitude, longitude, postalcode, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['PostalCode'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '[' + postalcode +']: ' + neighborhood + ', ' + borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapToronto)  
    
mapToronto

### Toronto Static Map of Neighborhoods
![Toronto Static Map of Neighborhoods](Toronto_Neighborhood_Map_001.png "Toronto Static Map of Neighborhoods")

## Step 3: Explore Toronto Neighborhoods

In [22]:
# Create a function to collect nearby venues from all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):

    # Count the number of neighborhoods being explored
    countNeighborhoods = 0

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # Update the user on which neighborhood is being explored
        countNeighborhoods += 1
        print('{} of {}: Exploring venues nearby {}.'.format(countNeighborhoods, len(names), name))

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            FOURSQUARE_CLIENT_ID, 
            FOURSQUARE_CLIENT_SECRET, 
            FOURSQUARE_VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # Continue if Error with processing the GET request
        try:
            # make the GET request
            results = requests.get(url, timeout=10).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except: 
          pass        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Limit the number of neighborhoods that can be explored
MAX_NEIGHBORHOODS = 300

# Select which neighborhoods to explore
#selectNeighborhoods = neighborhoods.loc[neighborhoods['Borough'] == 'Downtown Toronto']
#selectNeighborhoods = neighborhoods.loc[neighborhoods['Borough'].str.contains("Toronto")]
selectNeighborhoods = neighborhoods.copy()

# Call Foursquare to explore nearby venues in each Neighborhood
venuesToronto = getNearbyVenues(names=selectNeighborhoods.head(MAX_NEIGHBORHOODS)['Neighborhood'],
                                   latitudes=selectNeighborhoods.head(MAX_NEIGHBORHOODS)['Latitude'],
                                   longitudes=selectNeighborhoods.head(MAX_NEIGHBORHOODS)['Longitude']
                                  )

1 of 209: Exploring venues nearby Parkwoods.
2 of 209: Exploring venues nearby Victoria Village.
3 of 209: Exploring venues nearby Regent Park.
4 of 209: Exploring venues nearby Lawrence Heights.
5 of 209: Exploring venues nearby Lawrence Manor.
6 of 209: Exploring venues nearby Queen's Park.
7 of 209: Exploring venues nearby Islington Avenue.
8 of 209: Exploring venues nearby Rouge.
9 of 209: Exploring venues nearby Malvern.
10 of 209: Exploring venues nearby Don Mills North.
11 of 209: Exploring venues nearby Woodbine Gardens.
12 of 209: Exploring venues nearby Parkview Hill.
13 of 209: Exploring venues nearby Ryerson.
14 of 209: Exploring venues nearby Garden District.
15 of 209: Exploring venues nearby Glencairn.
16 of 209: Exploring venues nearby Cloverdale.
17 of 209: Exploring venues nearby Islington.
18 of 209: Exploring venues nearby Martin Grove.
19 of 209: Exploring venues nearby Princess Gardens.
20 of 209: Exploring venues nearby West Deane Park.
21 of 209: Exploring venue

165 of 209: Exploring venues nearby Harbourfront West.
166 of 209: Exploring venues nearby King and Spadina.
167 of 209: Exploring venues nearby Railway Lands.
168 of 209: Exploring venues nearby South Niagara.
169 of 209: Exploring venues nearby Humber Bay Shores.
170 of 209: Exploring venues nearby Mimico South.
171 of 209: Exploring venues nearby New Toronto.
172 of 209: Exploring venues nearby Albion Gardens.
173 of 209: Exploring venues nearby Beaumond Heights.
174 of 209: Exploring venues nearby Humbergate.
175 of 209: Exploring venues nearby Jamestown.
176 of 209: Exploring venues nearby Mount Olive.
177 of 209: Exploring venues nearby Silverstone.
178 of 209: Exploring venues nearby South Steeles.
179 of 209: Exploring venues nearby Thistletown.
180 of 209: Exploring venues nearby L'Amoreaux West.
181 of 209: Exploring venues nearby Rosedale.
182 of 209: Exploring venues nearby Stn A PO Boxes 25 The Esplanade.
183 of 209: Exploring venues nearby Alderwood.
184 of 209: Exploring

In [24]:
# Check how many venues were found in each neighborhood
venuesToronto.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                               
Adelaide                                                             100   
Agincourt                                                             16   
Agincourt North                                                       31   
Albion Gardens                                                         6   
Alderwood                                                              5   
Bathurst Manor                                                         4   
Bathurst Quay                                                         25   
Bayview Village                                                        3   
Beaumond Heights                                                       7   
Bedford Park                                                          37   
Berczy Park                                                          100   
Birch Cliff                                                            2   
Bloordale Gardens                                                      4   
Brockton                                                              43   
Business Reply Mail Processing Centre 969 Eastern                     25   
CFB Toronto                                                          100   
CN Tower                                                              81   
Cabbagetown                                                           61   
Caledonia-Fairbanks                                                    5   
Canada Post Gateway Processing Centre                                100   
Cedarbrae                                                              4   
Central Bay Street                                                   100   
Chinatown                                                            100   
Christie                                                              13   
Church and Wellesley                                                  82   
Clairlea                                                               6   
Clarks Corners                                                         8   
Cliffcrest                                                            10   
Cliffside                                                              9   
Cliffside West                                                         9   
...                                                                  ...   
The Kingsway                                                          21   
The Queensway East                                                     7   
The Queensway West                                                     7   
Thistletown                                                            3   
Thorncliffe Park                                                      18   
Toronto Dominion Centre                                              100   
Toronto Islands                                                       14   
Trinity                                                              100   
Underground city                                                     100   
Union Station                                                        100   
University of Toronto                                                 65   
Upper Rouge                                                            2   
Upwood Park                                                            4   
Victoria Hotel                                                       100   
Victoria Village                                                       2   
West Deane Park                                                        4   
West Hill                                                              6   
Westmount                                                              4   
Weston                                                                 7   
Wexford                                                                2   
Wexford Heights        

In [25]:
# Check how many unique categories were found
print('There are {} uniques categories.'.format(len(venuesToronto['Venue Category'].unique())))

There are 317 uniques categories.


In [26]:
# Prepare to Cluster the data
onehotToronto = pd.get_dummies(venuesToronto[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
onehotToronto['Neighborhood'] = venuesToronto['Neighborhood'] 

# Move neighborhood column to the first column
onehotColumns = [onehotToronto.columns[-1]] + list(onehotToronto.columns[:-1])
onehotToronto = onehotToronto[onehotColumns]

# Check what the data looks like
print( onehotToronto.shape )
onehotToronto.head()

(4878, 317)


Yoga Studio  Accessories Store  Adult Boutique  Afghan Restaurant  \
0            0                  0               0                  0   
1            0                  0               0                  0   
2            0                  0               0                  0   
3            0                  0               0                  0   
4            0                  0               0                  0   

   African Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                   0        0                   0               0   
1                   0        0                   0               0   
2                   0        0                   0               0   
3                   0        0                   0               0   
4                   0        0                   0               0   

   Airport Service  Airport Terminal  ...  University  \
0                0                 0  ...           0   
1                0                 0  ...           0   
2                0                 0  ...           0   
3                0                 0  ...           0   
4                0                 0  ...           0   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  \
0                      0                0         0          0            0   
1                      0                0         0          0            0   
2                      0                0         0          0            0   
3                      0                0         0          0            0   
4                      0                0         0          0            0   

   Women's Store  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 317 columns]

In [27]:
# Group the Explored Venues by Neighborhood to see the percentage of each Venue Category

groupedToronto = onehotToronto.groupby('Neighborhood').mean().reset_index()
groupedToronto

Neighborhood  Yoga Studio  \
0                                             Adelaide     0.020000   
1                                            Agincourt     0.000000   
2                                      Agincourt North     0.000000   
3                                       Albion Gardens     0.000000   
4                                            Alderwood     0.000000   
5                                       Bathurst Manor     0.000000   
6                                        Bathurst Quay     0.000000   
7                                      Bayview Village     0.000000   
8                                     Beaumond Heights     0.000000   
9                                         Bedford Park     0.000000   
10                                         Berczy Park     0.000000   
11                                         Birch Cliff     0.000000   
12                                   Bloordale Gardens     0.000000   
13                                            Brockton     0.000000   
14   Business Reply Mail Processing Centre 969 Eastern     0.040000   
15                                         CFB Toronto     0.000000   
16                                            CN Tower     0.012346   
17                                         Cabbagetown     0.000000   
18                                 Caledonia-Fairbanks     0.000000   
19               Canada Post Gateway Processing Centre     0.000000   
20                                           Cedarbrae     0.000000   
21                                  Central Bay Street     0.010000   
22                                           Chinatown     0.000000   
23                                            Christie     0.000000   
24                                Church and Wellesley     0.000000   
25                                            Clairlea     0.000000   
26                                      Clarks Corners     0.000000   
27                                          Cliffcrest     0.000000   
28                                           Cliffside     0.000000   
29                                      Cliffside West     0.000000   
..                                                 ...          ...   
176                                       The Kingsway     0.000000   
177                                 The Queensway East     0.000000   
178                                 The Queensway West     0.000000   
179                                        Thistletown     0.000000   
180                                   Thorncliffe Park     0.000000   
181                            Toronto Dominion Centre     0.000000   
182                                    Toronto Islands     0.000000   
183                                            Trinity     0.000000   
184                                   Underground city     0.000000   
185                                      Union Station     0.000000   
186                              University of Toronto     0.000000   
187                                        Upper Rouge     0.000000   
188                                        Upwood Park     0.000000   
189                                     Victoria Hotel     0.000000   
190                                   Victoria Village     0.000000   
191                                    West Deane Park     0.000000   
192                                          West Hill     0.000000   
193                                          Westmount     0.000000   
194                                             Weston     0.000000   
195                                            Wexford     0.000000   
196                                    Wexford Heights     0.000000   
197                                    Willowdale West     0.000000   
198                                     Wilson Heights     0.000000   
199                                             Woburn     0.000000   
200                                   Woodbine Gardens     0.00000

In [29]:
# Limit the number of neighborhoods that can be shown
MAX_NEIGHBORHOODS = 5

# Print each neighborhood along with the top 10 most common venues
COUNT_TOP_VENUES = 10

for hood in groupedToronto['Neighborhood'].head(MAX_NEIGHBORHOODS):
    print("----"+hood+"----")
    temp = groupedToronto[groupedToronto['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(COUNT_TOP_VENUES))
    print('\n')

----Adelaide----
                    venue  freq
0             Coffee Shop  0.06
1      Italian Restaurant  0.06
2              Restaurant  0.05
3                     Bar  0.05
4                     Spa  0.04
5  Furniture / Home Store  0.04
6                  Bakery  0.03
7                    Café  0.03
8                     Gym  0.03
9             Yoga Studio  0.02


----Agincourt----
                    venue  freq
0      Chinese Restaurant  0.31
1        Asian Restaurant  0.12
2             Coffee Shop  0.06
3              Food Court  0.06
4       Korean Restaurant  0.06
5    Cantonese Restaurant  0.06
6              Restaurant  0.06
7  Peking Duck Restaurant  0.06
8                  Bakery  0.06
9    Hong Kong Restaurant  0.06


----Agincourt North----
                venue  freq
0              Bakery  0.06
1  Chinese Restaurant  0.06
2      Ice Cream Shop  0.06
3           Juice Bar  0.03
4       Movie Theater  0.03
5                Bank  0.03
6      Clothing Store  0.03
7        

In [30]:
# Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# Create a new DataFrame with the Top-10 Venues for each Neighborhood
COUNT_TOP_VENUES = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(COUNT_TOP_VENUES):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sortedNeighborhoodVenues = pd.DataFrame(columns=columns)
sortedNeighborhoodVenues['Neighborhood'] = groupedToronto['Neighborhood']

for ind in np.arange(groupedToronto.shape[0]):
    sortedNeighborhoodVenues.iloc[ind, 1:] = return_most_common_venues(groupedToronto.iloc[ind, :], COUNT_TOP_VENUES)

sortedNeighborhoodVenues.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0         Adelaide    Italian Restaurant           Coffee Shop   
1        Agincourt    Chinese Restaurant      Asian Restaurant   
2  Agincourt North                Bakery    Chinese Restaurant   
3   Albion Gardens        Baseball Field   Filipino Restaurant   
4        Alderwood           Pizza Place              Pharmacy   

  3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
0            Restaurant                   Bar                    Spa   
1            Food Court                Bakery  Vietnamese Restaurant   
2        Ice Cream Shop           Coffee Shop             Beer Store   
3           Gas Station          Liquor Store             Beer Store   
4           Coffee Shop          Dance Studio      Convenience Store   

    6th Most Common Venue        7th Most Common Venue 8th Most Common Venue  \
0  Furniture / Home Store                         Café                Bakery   
1  Peking Duck Restaurant                  Coffee Shop  Hong Kong Restaurant   
2              Taco Place                  Supermarket                  Bank   
3                    Bank                Women's Store      Doner Restaurant   
4           Women's Store  Eastern European Restaurant       Doctor's Office   

  9th Most Common Venue 10th Most Common Venue  
0                   Gym   Caribbean Restaurant  
1  Cantonese Restaurant             Restaurant  
2   Japanese Restaurant    Sporting Goods Shop  
3            Donut Shop              Drugstore  
4               Dog Run       Doner Restaurant

## Step 4: Cluster Toronto Neighborhoods

In [50]:
# Run k-means to cluster the neighborhood into clusters
COUNT_K_MEANS_CLUSTERS = 4

clusteredTorontoGroups = groupedToronto.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=COUNT_K_MEANS_CLUSTERS, random_state=0).fit(clusteredTorontoGroups)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:300]

array([2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0,
       0, 0, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3,
       2, 1, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1, 2, 2, 1, 3,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 3, 3,
       2, 2, 2, 2, 1, 1, 2, 2])

In [51]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# Add clustering labels
labeledSortedNeighborhoodVenues = sortedNeighborhoodVenues.copy()

labeledSortedNeighborhoodVenues.insert(0, 'Cluster Labels', kmeans.labels_)

# Extend the original Neighborhood Data
mergedToronto = selectNeighborhoods.copy()

# Merge the grouped data with the neighborhood data to add latitude/longitude for each neighborhood
mergedToronto = mergedToronto.join(labeledSortedNeighborhoodVenues.set_index('Neighborhood'), on='Neighborhood')

mergedToronto.head() # Check the merged columns

PostalCode           Borough      Neighborhood                     URL  \
0        M3A        North York         Parkwoods         /wiki/Parkwoods   
1        M4A        North York  Victoria Village  /wiki/Victoria_Village   
2        M5A  Downtown Toronto       Regent Park       /wiki/Regent_Park   
3        M6A        North York  Lawrence Heights  /wiki/Lawrence_Heights   
4        M6A        North York    Lawrence Manor    /wiki/Lawrence_Manor   

                                Title   Latitude  Longitude  Cluster Labels  \
0         Parkwoods, Toronto, Ontario  43.686575 -79.409993             1.0   
1  Victoria Village, Toronto, Ontario  43.731540 -79.314280             3.0   
2       Regent Park, Toronto, Ontario  43.660690 -79.360310             2.0   
3  Lawrence Heights, Toronto, Ontario  43.723570 -79.437110             3.0   
4    Lawrence Manor, Toronto, Ontario  43.722920 -79.431310             2.0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                  Park                  Café            Bagel Shop   
1            Food Stand                  Park         Women's Store   
2           Coffee Shop       Thai Restaurant             Pet Store   
3                  Park     Accessories Store         Women's Store   
4         Women's Store      Department Store          Liquor Store   

           4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0  Vegetarian / Vegan Restaurant             Juice Bar      Sushi Restaurant   
1                          Diner              Dive Bar       Doctor's Office   
2                 Animal Shelter     Electronics Store  Fast Food Restaurant   
3    Eastern European Restaurant              Dive Bar       Doctor's Office   
4             Mexican Restaurant            Shoe Store     Electronics Store   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0           Coffee Shop          Burger Joint    Italian Restaurant   
1               Dog Run      Doner Restaurant            Donut Shop   
2            Beer Store      Sushi Restaurant            Food Truck   
3               Dog Run      Doner Restaurant            Donut Shop   
4            Kids Store              Bus Stop           Supermarket   

  10th Most Common Venue  
0    Japanese Restaurant  
1              Drugstore  
2             Restaurant  
3              Drugstore  
4         Sandwich Place

### Map of Clusters

In [52]:
# Visualize the Clusters

# Create a map
map_clusters = folium.Map(location=[locationToronto.latlng[0], locationToronto.latlng[1]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(COUNT_K_MEANS_CLUSTERS)
ys = [i + x + (i*x)**2 for i in range(COUNT_K_MEANS_CLUSTERS)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mergedToronto['Latitude'], mergedToronto['Longitude'], mergedToronto['Neighborhood'], mergedToronto['Cluster Labels']):
    try:
        idCluster = int(cluster)
    except:
        idCluster = 0
    label = folium.Popup(str(poi) + ' Cluster ' + str(idCluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[idCluster-1],
        fill=True,
        fill_color=rainbow[idCluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Toronto Static Map of Neighborhood Clusters
![Toronto Static Map of Neighborhood Clusters](Toronto_Neighborhood_Map_002.png "Toronto Static Map of Neighborhood Clusters")

## Step 5: Examine Toronto Clusters

### Cluster 0

In [55]:
mergedToronto.loc[mergedToronto['Cluster Labels'] == 0, mergedToronto.columns[[1] + list(range(5, mergedToronto.shape[1]))]].head(10)

Borough   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
44   North York  43.720190 -79.499915             0.0           Pizza Place   
69   North York  43.720190 -79.499915             0.0           Pizza Place   
82   North York  43.720190 -79.499915             0.0           Pizza Place   
87   North York  43.720190 -79.499915             0.0           Pizza Place   
96   North York  43.720190 -79.499915             0.0           Pizza Place   
109  North York  43.720190 -79.499915             0.0           Pizza Place   
175   Etobicoke  43.747074 -79.594665             0.0           Pizza Place   

    2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
44               Pharmacy  Vietnamese Restaurant                Bakery   
69               Pharmacy  Vietnamese Restaurant                Bakery   
82               Pharmacy  Vietnamese Restaurant                Bakery   
87               Pharmacy  Vietnamese Restaurant                Bakery   
96               Pharmacy  Vietnamese Restaurant                Bakery   
109              Pharmacy  Vietnamese Restaurant                Bakery   
175         Women's Store    Dumpling Restaurant        Discount Store   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
44    Dumpling Restaurant              Dive Bar       Doctor's Office   
69    Dumpling Restaurant              Dive Bar       Doctor's Office   
82    Dumpling Restaurant              Dive Bar       Doctor's Office   
87    Dumpling Restaurant              Dive Bar       Doctor's Office   
96    Dumpling Restaurant              Dive Bar       Doctor's Office   
109   Dumpling Restaurant              Dive Bar       Doctor's Office   
175              Dive Bar       Doctor's Office               Dog Run   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
44                Dog Run      Doner Restaurant             Donut Shop  
69                Dog Run      Doner Restaurant             Donut Shop  
82                Dog Run      Doner Restaurant             Donut Shop  
87                Dog Run      Doner Restaurant             Donut Shop  
96                Dog Run      Doner Restaurant             Donut Shop  
109               Dog Run      Doner Restaurant             Donut Shop  
175      Doner Restaurant            Donut Shop              Drugstore

### Cluster 1

In [56]:
mergedToronto.loc[mergedToronto['Cluster Labels'] == 1, mergedToronto.columns[[1] + list(range(5, mergedToronto.shape[1]))]].head(10)

Borough   Latitude  Longitude  Cluster Labels  \
0         North York  43.686575 -79.409993             1.0   
7        Scarborough  43.807660 -79.174050             1.0   
19         Etobicoke  43.652970 -79.557420             1.0   
20       Scarborough  43.789480 -79.176140             1.0   
29         Etobicoke  43.661689 -79.578263             1.0   
34       Scarborough  43.763430 -79.178200             1.0   
37              York  43.688570 -79.454830             1.0   
41  Downtown Toronto  43.673016 -79.422080             1.0   
43        North York  43.763780 -79.454770             1.0   
55        North York  43.693643 -79.401812             1.0   

   1st Most Common Venue 2nd Most Common Venue       3rd Most Common Venue  \
0                   Park                  Café                  Bagel Shop   
7            Pizza Place           Bus Station                 Candy Store   
19                  Park     Convenience Store               Movie Theater   
20                  Park              Pharmacy                 Bus Station   
29           Pizza Place           Coffee Shop                        Park   
34     Convenience Store              Tea Room  Construction & Landscaping   
37         Women's Store                Market                    Pharmacy   
41                  Park           Coffee Shop                 Candy Store   
43            Playground                  Park           Convenience Store   
55                 Hotel  Gym / Fitness Center                Skating Rink   

            4th Most Common Venue        5th Most Common Venue  \
0   Vegetarian / Vegan Restaurant                    Juice Bar   
7                            Park                  Event Space   
19                          Hotel                Women's Store   
20                 Baseball Field  Eastern European Restaurant   
29              Convenience Store           Chinese Restaurant   
34                           Park                   Restaurant   
37                           Park         Fast Food Restaurant   
41                           Café           Italian Restaurant   
43                 Baseball Field                     Dive Bar   
55                           Park                Women's Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0       Sushi Restaurant           Coffee Shop          Burger Joint   
7             Donut Shop              Dive Bar       Doctor's Office   
19       Doctor's Office               Dog Run      Doner Restaurant   
20       Doctor's Office               Dog Run      Doner Restaurant   
29         Event Service   Dumpling Restaurant              Dive Bar   
34         Women's Store             Drugstore              Dive Bar   
37                   Gym          Dessert Shop                 Diner   
41            Beer Store             Nightclub   Japanese Restaurant   
43       Doctor's Office               Dog Run      Doner Restaurant   
55        Discount Store              Dive Bar       Doctor's Office   

   9th Most Common Venue 10th Most Common Venue  
0     Italian Restaurant    Japanese Restaurant  
7     Falafel Restaurant                Exhibit  
19            Donut Shop              Drugstore  
20            Donut Shop              Drugstore  
29       Doctor's Office                Dog Run  
34       Doctor's Office                Dog Run  
37        Discount Store               Dive Bar  
41                 Diner          Grocery Store  
43            Donut Shop              Drugstore  
55               Dog Run       Doner Restaurant

### Cluster 2

In [57]:
mergedToronto.loc[mergedToronto['Cluster Labels'] == 2, mergedToronto.columns[[1] + list(range(5, mergedToronto.shape[1]))]].head(10)

Borough   Latitude  Longitude  Cluster Labels  \
2   Downtown Toronto  43.660690 -79.360310             2.0   
4         North York  43.722920 -79.431310             2.0   
5       Queen's Park  43.662530 -79.390170             2.0   
6          Etobicoke  43.722687 -79.558689             2.0   
8        Scarborough  43.809770 -79.220840             2.0   
9         North York  43.699339 -79.337722             2.0   
10         East York  43.705570 -79.300590             2.0   
12  Downtown Toronto  43.648829 -79.402486             2.0   
13  Downtown Toronto  43.657940 -79.375620             2.0   
14        North York  43.713271 -79.424630             2.0   

   1st Most Common Venue 2nd Most Common Venue   3rd Most Common Venue  \
2            Coffee Shop       Thai Restaurant               Pet Store   
4          Women's Store      Department Store            Liquor Store   
5            Coffee Shop                   Gym        Sushi Restaurant   
6            Coffee Shop     Convenience Store          Clothing Store   
8               Pharmacy           Pizza Place          Sandwich Place   
9     Seafood Restaurant            Public Art           Women's Store   
10           Wings Joint           Bus Station             Coffee Shop   
12           Coffee Shop                   Bar  Furniture / Home Store   
13           Coffee Shop                 Hotel                    Café   
14                  Café              Pharmacy               Pet Store   

        4th Most Common Venue  5th Most Common Venue 6th Most Common Venue  \
2              Animal Shelter      Electronics Store  Fast Food Restaurant   
4          Mexican Restaurant             Shoe Store     Electronics Store   
5          Persian Restaurant  Portuguese Restaurant        Sandwich Place   
6               Women's Store      Electronics Store       Doctor's Office   
8        Gym / Fitness Center          Grocery Store  Fast Food Restaurant   
9         Dumpling Restaurant               Dive Bar       Doctor's Office   
10                 Restaurant          Grocery Store   Dumpling Restaurant   
12                Pizza Place                   Café          Dessert Shop   
13  Middle Eastern Restaurant            Pizza Place        Sandwich Place   
14               Dessert Shop                   Bank   Dumpling Restaurant   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
2             Beer Store      Sushi Restaurant            Food Truck   
4             Kids Store              Bus Stop           Supermarket   
5                    Bar    Italian Restaurant                  Café   
6                Dog Run      Doner Restaurant            Donut Shop   
8      Convenience Store          Skating Rink       Bubble Tea Shop   
9                Dog Run      Doner Restaurant            Donut Shop   
10              Dive Bar       Doctor's Office               Dog Run   
12                  Park      Ramen Restaurant      Asian Restaurant   
13  Fast Food Restaurant                 Diner        Clothing Store   
14       Doctor's Office               Dog Run      Doner Restaurant   

   10th Most Common Venue  
2              Restaurant  
4          Sandwich Place  
5                 Theater  
6               Drugstore  
8              Donut Shop  
9               Drugstore  
10       Doner Restaurant  
12     Italian Restaurant  
13    Japanese Restaurant  
14             Donut Shop

### Cluster 3

In [58]:
mergedToronto.loc[mergedToronto['Cluster Labels'] == 3, mergedToronto.columns[[1] + list(range(5, mergedToronto.shape[1]))]].head(10)

Borough   Latitude  Longitude  Cluster Labels  \
1         North York  43.731540 -79.314280             3.0   
3         North York  43.723570 -79.437110             3.0   
22       Scarborough  43.778970 -79.131090             3.0   
30         Etobicoke  43.633910 -79.569480             3.0   
32       Scarborough  43.749530 -79.189920             3.0   
39         East York  43.700239 -79.351065             3.0   
42        North York  43.803030 -79.353460             3.0   
48  Downtown Toronto  43.706307 -79.514355             3.0   
65       Scarborough  43.736290 -79.273200             3.0   
94        North York  43.787300 -79.409830             3.0   

   1st Most Common Venue                     2nd Most Common Venue  \
1             Food Stand                                      Park   
3                   Park                         Accessories Store   
22                  Park                             Train Station   
30                  Park                             Women's Store   
32                  Park                                     Hotel   
39                  Park                         Indian Restaurant   
42                  Park  Residential Building (Apartment / Condo)   
48     Convenience Store                                      Park   
65                  Park                             Women's Store   
94                  Park                             Women's Store   

   3rd Most Common Venue        4th Most Common Venue 5th Most Common Venue  \
1          Women's Store                        Diner              Dive Bar   
3          Women's Store  Eastern European Restaurant              Dive Bar   
22         Women's Store               Discount Store              Dive Bar   
30    Dim Sum Restaurant               Discount Store              Dive Bar   
32         Women's Store                     Dive Bar       Doctor's Office   
39                Bridge                Women's Store              Dive Bar   
42         Women's Store               Discount Store              Dive Bar   
48         Women's Store          Dumpling Restaurant              Dive Bar   
65    Dim Sum Restaurant               Discount Store              Dive Bar   
94    Dim Sum Restaurant               Discount Store              Dive Bar   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
1        Doctor's Office               Dog Run      Doner Restaurant   
3        Doctor's Office               Dog Run      Doner Restaurant   
22       Doctor's Office               Dog Run      Doner Restaurant   
30       Doctor's Office               Dog Run      Doner Restaurant   
32               Dog Run      Doner Restaurant            Donut Shop   
39       Doctor's Office               Dog Run      Doner Restaurant   
42       Doctor's Office               Dog Run      Doner Restaurant   
48       Doctor's Office               Dog Run      Doner Restaurant   
65       Doctor's Office               Dog Run      Doner Restaurant   
94       Doctor's Office               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
1             Donut Shop              Drugstore  
3             Donut Shop              Drugstore  
22            Donut Shop              Drugstore  
30            Donut Shop              Drugstore  
32             Drugstore    Dumpling Restaurant  
39            Donut Shop              Drugstore  
42            Donut Shop              Drugstore  
48            Donut Shop              Drugstore  
65            Donut Shop              Drugstore  
94            Donut Shop              Drugstore

## The End.